In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../mats/2023-nwbc-data.csv')

In [3]:
df.shape

(736626, 95)

In [4]:
df['is_strike'] = 0
strike_events = ['called_strike', 'swinging_strike',
       'foul', 'blocked_ball', 'swinging_strike_blocked', 'hit_by_pitch',
       'foul_tip', 'foul_bunt', 'missed_bunt', 'bunt_foul_tip']
df.loc[df['description'].isin(strike_events), 'is_strike'] = 1

_df = df.loc[:, ['pitcher_name', 'pitch_name', 'is_strike']].groupby(['pitcher_name','pitch_name']).agg(n_pitches = ('is_strike','count')).reset_index()
df2 = df.merge(_df, on=['pitcher_name','pitch_name'])
df2 = df2.loc[df2['n_pitches']>=100]

df2['runs_scored'] = df2['post_bat_score'] - df2['bat_score']

In [5]:
df2.shape

(672545, 98)

In [6]:
df2.iloc[:,-2:].describe()

,n_pitches,runs_scored
count,672545.000000,672545.000000
mean,563.028991,0.028695
std,354.677992,0.209070
min,100.000000,0.000000
25%,290.000000,0.000000
50%,482.000000,0.000000
75%,754.000000,0.000000
max,1848.000000,4.000000


In [14]:
era=(df2
 .loc[:,['pitcher_name','pitcher','runs_scored']]
 .groupby(['pitcher_name','pitcher'],as_index=False)
 .sum())

#Important to know this takes filtered data and counts partial innings as full innings
_df = df2.loc[:,['pitcher','game_pk','inning']].drop_duplicates()
_df = (_df['pitcher']
       .value_counts()
       .to_frame().reset_index()
       .rename(columns={'index':'pitcher'
                        , 'pitcher':'n_innings'}))

era = era.merge(_df, on=['pitcher'])
era['era'] = (era['runs_scored']/era['n_innings'])*9
era.head()

,pitcher_name,pitcher,runs_scored,n_innings,era
0,"Abbott, Cory",676265,25,52,4.326923
1,"Abreu, Albert",656061,12,51,2.117647
2,"Abreu, Bryan",650556,26,81,2.888889
3,"Acevedo, Domingo",642758,32,82,3.512195
4,"Adam, Jason",592094,14,77,1.636364
